<a href="https://colab.research.google.com/github/FreddyPinto/recsys-steam-games/blob/feature/notebooks/2.1-feature-engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Engineering

El objetivo de este notebook es optimizar el rendimiento de la API y mejorar el rendimiento predictivo del modelo de Machine Learning para nuestro sistema de recomendación, con el fin de reducir las necesidades computacionales o de datos y mejorar la interpretabilidad de los resultados.

Para ello, haremos uso de la ingeniería de características, que consiste en crear, seleccionar o transformar las variables que se usarán para los endpoints que se consumirán en la API y el modelo.

Algunas de las tareas que realizaremos son:

- Determinar qué características son las más importantes.

- Aplicar un análisis de sentimiento con NLP.

- Codificar categóricos de alta cardinalidad.

- Reducir la dimensionalidad de los datos aplicando análisis de componentes principales para conservar la mayor parte de la información útil.

- Obtener una base de datos única y más eficiente para cada endpoint.

## 0 Configuraciones Globales e Importaciones

En esta sección, importamos todas las bibliotecas y/o modulos necesarios para nuestro proceso de feature engineering y establecemos configuraciones globales de ser requerido.

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import scipy as sp
import textblob
import sklearn
from textblob import TextBlob
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

print(f"System version: {sys.version}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"scipy version: {sp.__version__}")
print(f"textblob version: {textblob.__version__}")
print(f"scikit-learn version: {sklearn.__version__}")

System version: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
pandas version: 1.5.3
numpy version: 1.23.5
scipy version: 1.11.3
textblob version: 0.17.1
scikit-learn version: 1.2.2


## 1 Extracción

En esta sección, extraemos los datos de los archivos `steam_games`, `user_items` y `user_reviews` que estan en formato parquet.

### 1.1 Extracción de los datos

Creamos una función que lee cada archivo desde su directorio y lo carga a un DataFrame de `pandas`.

In [2]:
# Cargamos los archivos parquet
def read_parquet_files(parquet_files):
    dataframes = {}
    for name in parquet_files:
        dataframes[name] = pd.read_parquet(f'{name}.parquet', engine='pyarrow')
    return dataframes

parquet_files = ['steam_games','user_items', 'user_reviews']
dataframes = read_parquet_files(parquet_files)

# Convertimos a df.
df_steam_games = dataframes['steam_games']
df_user_items = dataframes['user_items']
df_user_reviews = dataframes['user_reviews']

## 2 Análisis de sentimiento

En esta sección, enriqueceremos el dataset `user_reviews` con una nueva columna llamada `sentiment_analysis`. Esta columna contendrá el resultado de aplicar un análisis de sentimiento con NLP a las reseñas de los juegos escritas por los usuarios. De esta manera, podremos explorar la opinión de los usuarios sobre los diferentes juegos.

El análisis de sentimiento consiste en asignar una etiqueta numérica a cada reseña, según el tono o la actitud que expresa el texto. Usaremos la siguiente escala:

* 0: si la reseña es **negativa**, es decir, si el usuario muestra insatisfacción, disgusto o decepción con el juego.
* 1: si la reseña es **neutral**, es decir, si el usuario muestra indiferencia, objetividad o ausencia de emoción con el juego.
* 2: si la reseña es **positiva**, es decir, si el usuario muestra satisfacción, gusto o admiración con el juego.



### 2.1 Función `sentiment_analysis`

Para realizar el análisis de sentimiento con NLP a las reseñas de los juegos, crearemos una función usando la librería TextBlob que se considera facil de usar y muy intuitiva. Usaremos la polaridad que es una medida numérica que indica si el texto es negativo o positivo, según el tono o la actitud que expresa. La polaridad varía entre -1 y 1, donde -1 significa muy negativo, 0 significa neutro y 1 significa muy positivo.

In [3]:
def sentiment_analysis(review):
    # Si la reseña está ausente, retorna 1 (neutral)
    if pd.isnull(review):
        return 1

    # Calcula la polaridad de la reseña usando TextBlob
    polarity = TextBlob(review).sentiment.polarity

    # Retorna 0 (malo) si la polaridad es menor que 0, 2 (positivo) si la polaridad es mayor que 0, y 1 (neutral) en caso contrario
    if polarity < 0:
        return 0
    elif polarity > 0:
        return 2
    else:
        return 1

* Aplicamos la función a la columna `review`.

In [4]:
df_user_reviews['sentiment_analysis'] = df_user_reviews['review'].apply(sentiment_analysis)

* Veamos algunos ejemplos:

In [5]:
df_user_reviews[['review','sentiment_analysis']].sample(5)

,review,sentiment_analysis
23011,"There is only mixed reviews because of ""lack o...",1
48565,"Got down to the ruins level, with tons of good...",2
34714,A simple concept which is a solid ton of fun. ...,2
5068,This man knows what he is on about Dexaldem T...,2
38249,best game ever,2


### 2.2 Eliminación de la columna `review`

La nueva columna `sentiment_analysis` reemplazará a la columna `review` en el dataset `user_reviews`, para facilitar el trabajo de los modelos de machine learning y el análisis de datos.

In [7]:
df_user_reviews.drop('review', axis=1, inplace=True)
df_user_reviews.head()

,item_id,recommend,user_id,posted_year,sentiment_analysis
0,1250,True,76561197970982479,2011,2
1,22200,True,76561197970982479,2011,2
2,43110,True,76561197970982479,2011,2
3,251610,True,js41637,2014,2
4,227300,True,js41637,2013,0


## 3 Diseño y estructura de las bases de datos para los endpoints de la API

En esta sección, nuestro objetivo es crear diferentes dataset a modo de pseudo base de datos para las funciones que se usarán en los endpoints de la API. De esta manera, podremos acceder a los datos que necesitamos de forma rápida y eficiente, sin tener que cargar toda la información para así, optimizar el rendimiento de la API.

### 3.1 Endpoints 1 y 2

Estos endpoints comparten información en común, por lo que podemos crear un solo dataset para ambos.

#### 3.1.1 Endpoint 1

def **PlayTimeGenre( *`genero` : str* )**:
    Retorna `año` con mas horas jugadas para el género dado.
Ejemplo de retorno:

``` js
{
   "Año de lanzamiento con más horas jugadas para Género X": 2013
}
```



#### 3.1.2 Endpoint 2

+ def **UserForGenre( *`genero` : str* )**:
    Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

Ejemplo de retorno:
```js
{
   "Usuario con más horas jugadas para Género X":"us213ndjss09sdf",
   "Horas jugadas":[
      {
         "Año":2013,
         "Horas":203
      },
      {
         "Año":2012,
         "Horas":100
      },
      {
         "Año":2011,
         "Horas":23
      }
   ]
}
```

#### 3.1.4 Pseudo Database 1

Para crear un solo dataset que pueda ser utilizado como pseudo base de datos para estos endpoints, necesitamos combinar `df_steam_games` con `df_user_items` de tal manera que tengamos toda la información necesaria en un solo lugar. Para esto solo necesitamos las columnas:
`item_id`,`genres`,`release_year` del DataFrame `steam_games`. También `item_id`, `user_id` y `playtime_forever` del DataFrame `user_items`.

* Primero, seleccionamos solo las columnas necesarias:

In [8]:
steam_games_columns = ['item_id','genres','release_year']
user_items_columns = ['item_id','user_id', 'playtime_forever']

* Segundo, creamos subsets de los DataFrames con solo las columnas necesarias:

In [9]:
df_games_subset = df_steam_games[steam_games_columns]
df_items_subset = df_user_items[user_items_columns]

* Luego, hacemos un merge entre `steam_games` y `user_items` en la columna `item_id`.

In [10]:
df_pseudo_db1 = pd.merge(df_games_subset, df_items_subset, on='item_id')
df_pseudo_db1.head()

,item_id,genres,release_year,user_id,playtime_forever
0,282010,Racing,1997,UTNerd24,0.083333
1,282010,Racing,1997,I_DID_911_JUST_SAYING,0.000000
2,282010,Racing,1997,76561197962104795,0.000000
3,282010,Racing,1997,r3ap3r78,0.000000
4,282010,Racing,1997,saint556,0.216667


In [11]:
df_pseudo_db1.shape

(15255072, 5)

- Con el fin de ahorrar recursos, solo usaremos los registros de juegos que cumplan con las siguientes condiciones: tener un `release_year` válido, un `genres` con una popularidad alta tomando como referencia la frecuencia relativa y haber sido jugados al menos una vez.

In [12]:
# Calculamos la frecuencia relativa de cada género
df_pseudo_db1.genres.value_counts()/len(df_pseudo_db1)

Action                       0.229263
Adventure                    0.182320
Indie                        0.127814
Strategy                     0.098006
RPG                          0.087477
Simulation                   0.068177
Casual                       0.062629
Free to Play                 0.058383
Massively Multiplayer        0.051663
Racing                       0.010949
Sports                       0.008968
Early Access                 0.008278
Education                    0.004132
Utilities                    0.000933
Video Production             0.000321
Web Publishing               0.000259
Software Training            0.000170
unknown                      0.000129
Audio Production             0.000077
Photo Editing                0.000039
Animation &amp; Modeling     0.000009
Design &amp; Illustration    0.000005
Name: genres, dtype: float64

- De acuerdo con la observación anterior, seleccionamos los 10 géneros más frecuentes, excluyendo las categorías que no consideramos un género como tal, sino que hacen referencia a por ejemplo si el juego es gratis (`Free to Play`) o tiene un acceso temprano (`Early Access`).

In [13]:
# Seleccionamos los 10 géneros mas frecuentes
top_10_popular_genres = ['Action', 'Adventure', 'Indie', 'Strategy', 'RPG', 'Simulation', 'Casual', 'Massively Multiplayer', 'Racing', 'Sports']

# Filtramos por las condiciones establecidas
df_pseudo_db1 = df_pseudo_db1[(df_pseudo_db1['release_year'] != 'unknown') & (df_pseudo_db1['playtime_forever'] > 0) & (df_pseudo_db1['genres'].isin(top_10_popular_genres))].reset_index(drop=True)
df_pseudo_db1.shape

(9753061, 5)

- Para optimizar el uso de la memoria en el deploy, convertiremos las columnas a los tipos de datos adecuados según su contenido.

In [14]:
df_pseudo_db1['release_year'] = df_pseudo_db1['release_year'].astype('int16')
df_pseudo_db1['playtime_forever'] = df_pseudo_db1['playtime_forever'].astype('float32')
df_pseudo_db1.memory_usage(deep=True)

Index                     128
item_id             609650394
genres              629483569
release_year         19506122
user_id             686349338
playtime_forever     39012244
dtype: int64

* Por último, creamos una tabla pivote que tenga como índice `user_id` y `release_year`, como columnas `genres` y como valores únicos la suma de `playtime_forever`.

In [15]:
df_pseudo_db1 = df_pseudo_db1.pivot_table(index=['user_id', 'release_year'], columns='genres', values='playtime_forever', aggfunc='sum', fill_value=0)
df_pseudo_db1

genres                        Action   Adventure     Casual      Indie  \
user_id    release_year                                                  
--000--    2006            15.416667   15.416667   0.000000  15.416667   
           2009            88.816666   88.816666   0.000000   0.000000   
           2010             0.366667    0.000000   0.000000   0.366667   
           2011           108.699997  108.699997   0.000000  30.616665   
           2012          1822.516724   37.150002  30.016666  37.700001   
...                              ...         ...        ...        ...   
zzzmidmiss 2010             7.783334    0.166667   3.916667   7.950000   
           2011            38.366665   38.366665   1.250000   1.750000   
           2012            98.366669   61.650005   6.083333  51.316666   
           2013             1.633333    1.750000   0.283333   1.750000   
           2014             0.266667    0.266667   0.266667   0.000000   

genres                   Massively Multiplayer        RPG     Racing  \
user_id    release_year                                                
--000--    2006                      15.416667   0.000000   0.000000   
           2009                       0.000000   0.000000   0.000000   
           2010                       0.000000   0.000000   0.000000   
           2011                      46.049999  62.649998  46.049999   
           2012                      10.500000  29.516666   0.000000   
...                                        ...        ...        ...   
zzzmidmiss 2010                       0.000000   0.000000   0.000000   
           2011                       0.266667  37.599998   0.266667   
           2012                       8.016666  45.500000   0.000000   
           2013                       0.166667   0.166667   0.000000   
           2014                       0.266667   0.000000   0.000000   

genres                   Simulation    Sports     Strategy  
user_id    release_year                                     
--000--    2006           15.416667  0.000000     0.000000  
           2009            0.000000  0.000000     0.000000  
           2010            0.000000  0.000000     0.000000  
           2011           11.083333  0.000000    11.083333  
           2012            0.000000  7.683333  1796.400024  
...                             ...       ...          ...  
zzzmidmiss 2010            3.233333  3.233333     3.400000  
           2011            0.000000  0.000000     1.150000  
           2012            6.450000  0.000000    15.383334  
           2013            0.000000  0.000000     1.466667  
           2014            0.266667  0.266667     0.266667  

[660003 rows x 10 columns]

In [16]:
df_pseudo_db1.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 660003 entries, ('--000--', 2006) to ('zzzmidmiss', 2014)
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Action                 660003 non-null  float32
 1   Adventure              660003 non-null  float32
 2   Casual                 660003 non-null  float32
 3   Indie                  660003 non-null  float32
 4   Massively Multiplayer  660003 non-null  float32
 5   RPG                    660003 non-null  float32
 6   Racing                 660003 non-null  float32
 7   Simulation             660003 non-null  float32
 8   Sports                 660003 non-null  float32
 9   Strategy               660003 non-null  float32
dtypes: float32(10)
memory usage: 28.8+ MB


### 3.2 Endpoints 3, 4 y 5.

#### 3.2.1 Endpoint 3

+ def **UsersRecommend( *`año` : int* )**:
   Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
  

Ejemplo de retorno:
```js
[
   {
      "Puesto 1":"X"
   },
   {
      "Puesto 2":"Y"
   },
   {
      "Puesto 3":"Z"
   }
]
```


#### 3.2.2 Endpoint 4

+ def **UsersWorstDeveloper( *`año` : int* )**:
   Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
  
Ejemplo de retorno:
```js
[
   {
      "Puesto 1":"X"
   },
   {
      "Puesto 2":"Y"
   },
   {
      "Puesto 3":"Z"
   }
]
```

#### 3.2.3 Endpoint 5

def **sentiment_analysis( *`empresa desarrolladora` : str* )**:
    Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total
    de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.

Ejemplo de retorno:
```js
{
   "Valve":[
      Negative = 182,
      Neutral = 120,
      Positive = 278
   ]
}
```


#### 3.1.4 Pseudo Database 2

- Para crear un solo dataset que pueda ser utilizado como pseudo base de datos para estos endpoints, necesitamos combinar `df_steam_games` con `df_user_reviews` de tal manera que tengamos toda la información necesaria en un solo lugar. Para esto solo necesitamos las columnas:
`item_id`,`item_name`,`developer` del DataFrame `steam_games`. También `item_id`, `recommend`, `sentiment_analysis` y `posted_year` del DataFrame `user_reviews`.

- Primero, seleccionamos las columnas necesarias:

In [17]:
steam_games_columns = ['item_id', 'item_name', 'developer']
user_reviews_columns = ['item_id', 'recommend','sentiment_analysis','posted_year']

* Segundo, creamos subsets de los DataFrames con solo las columnas necesarias:

In [18]:
df_games_subset = df_steam_games[steam_games_columns]
df_reviews_subset = df_user_reviews[user_reviews_columns]

* Luego, hacemos un merge entre los subsets `steam_games` y `user_reviews` en la columna `item_id`.

In [19]:
df_pseudo_db2 = pd.merge(df_games_subset, df_reviews_subset, on='item_id')
df_pseudo_db2.head()

,item_id,item_name,developer,recommend,sentiment_analysis,posted_year
0,282010,Carmageddon Max Pack,Stainless Games Ltd,True,1,unknown
1,282010,Carmageddon Max Pack,Stainless Games Ltd,True,1,unknown
2,282010,Carmageddon Max Pack,Stainless Games Ltd,True,1,unknown
3,70,Half-Life,Valve,True,0,2015
4,70,Half-Life,Valve,True,0,2011


- Con el objetivo de ahorrar recursos, filtraremos los registros de juegos que cumplan con las siguientes condiciones: tener un review con un `posted_year` válido y un `developer` conocido. Además, eliminaremos la columna `item_id`, ya que no la necesitamos.

In [20]:
df_pseudo_db2 = df_pseudo_db2[(df_pseudo_db2['posted_year'] != 'unknown') & (df_pseudo_db2['developer'] != 'unknown') ].reset_index(drop=True)
df_pseudo_db2.drop('item_id',axis=1, inplace=True)
df_pseudo_db2.head()

,item_name,developer,recommend,sentiment_analysis,posted_year
0,Half-Life,Valve,True,0,2015
1,Half-Life,Valve,True,0,2011
2,Half-Life,Valve,True,0,2014
3,Half-Life,Valve,True,2,2013
4,Half-Life,Valve,True,0,2013


- Por último, para optimizar el uso de la memoria en el deploy de la API, convertiremos las columnas a los tipos de datos adecuados según su contenido.

In [21]:
df_pseudo_db2['sentiment_analysis'] = df_pseudo_db2['sentiment_analysis'].astype('int8')
df_pseudo_db2['posted_year'] = df_pseudo_db2['posted_year'].astype('int16')
df_pseudo_db2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152619 entries, 0 to 152618
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   item_name           152619 non-null  object
 1   developer           152619 non-null  object
 2   recommend           152619 non-null  bool  
 3   sentiment_analysis  152619 non-null  int8  
 4   posted_year         152619 non-null  int16 
dtypes: bool(1), int16(1), int8(1), object(2)
memory usage: 2.9+ MB


## 4 Recommendation Engine / System

En esta sección, usaremos las estrategia de Collaborative Filtering para crear el **sistema de recomendación**. Para ello, se ofrecen dos propuestas:

1. User-based:
  * Se identifican usuarios similares
  * Se recomiendan nuevos ítems a otros usuarios basado en el rating dado por otros usuarios similares.

2. Item-based:
  * Calcular la similitud entre items
  * Encontrar los “mejores items similares” a los que un usuario no tenga evaluados y recomendárselos.

### 4.1 Calculando los Ratings

Como no tenemos **REALMENTE** una valoración o *rating* del 1 al 5 (como podríamos tener por ejemplo al valorar películas), se decide crear uno  a partir del análisis de sentimiento y las recomendaciones del usuario. Como criterio usaremos el análisis de sentimiento como el factor principal, y la recomendación como el factor secundario, para un rango del 1 al 5:

| sentiment_analysis | recommend | rating |
|--------------------|-----------|--------|
|    0 (negativo)    |	 False   |    1   |
|    0 (negativo)    |	 True    |    1   |
|    1 (neutral)     |	 False   |    2   |
|    1 (neutral)     |	 True    |    3   |
|    2 (positivo)    |	 False   |    4   |
|    2 (positivo)    |	 True    |    5   |


* Vamos a unir los tres conjuntos de datos, esto nos permitirá tener un solo conjunto de datos con toda la información relevante sobre los usuarios, como los juegos que han comprado o jugado, el análisis de sentimiento y la recomendación. Vamos a usar la función merge de pandas para hacer esta operación.

* Primero, seleccionamos solo las columnas necesarias:

In [22]:
steam_games_columns = ['item_id', 'item_name']
user_reviews_columns = ['item_id','user_id', 'recommend','sentiment_analysis']
user_items_columns = ['user_id','item_id']

* Segundo, creamos subsets de los DataFrames con solo las columnas necesarias:

In [23]:
df_games_subset = df_steam_games[steam_games_columns]
df_reviews_subset = df_user_reviews[user_reviews_columns]
df_items_subset = df_user_items[user_items_columns]

* Luego, hacemos un merge entre `steam_games` y `user_items` en la columna `item_id`.

In [24]:
df_user_games = pd.merge(df_games_subset, df_items_subset, on='item_id')
df_user_games.head()

,item_id,item_name,user_id
0,282010,Carmageddon Max Pack,UTNerd24
1,282010,Carmageddon Max Pack,I_DID_911_JUST_SAYING
2,282010,Carmageddon Max Pack,76561197962104795
3,282010,Carmageddon Max Pack,r3ap3r78
4,282010,Carmageddon Max Pack,saint556


* Hacemos otro merge esta vez entre `user_games` y `user_items` en la columnas `item_id` y `user_id`.

In [25]:
df_features = pd.merge(df_user_games, df_reviews_subset, on=["user_id", "item_id"])
df_features.drop_duplicates(inplace=True)
df_features.reset_index(drop=True, inplace=True)
df_features.head()

,item_id,item_name,user_id,recommend,sentiment_analysis
0,282010,Carmageddon Max Pack,InstigatorAU,True,1
1,70,Half-Life,EizanAratoFujimaki,True,0
2,70,Half-Life,GamerFag,True,0
3,70,Half-Life,76561198020928326,True,0
4,70,Half-Life,Bluegills,True,2


* Con el fin de reducir la dimensionalidad de los datos y conservar la mayor parte de la información útil para nuestro modelo, seleccionaremos solo los usuarios que hayan hecho al menos 5 reviews y los juegos que tengan 10 o más reviews.

In [26]:
# Agrupamos el dataframe por item_name para contar los juegos
count = df_features.groupby("item_name").size()

# Seleccionamos solo los juegos que tengan al menos 10 reviews.
df_features = df_features.loc[df_features["item_name"].isin(count[count >= 10].index), :]
df_features.head()


,item_id,item_name,user_id,recommend,sentiment_analysis
1,70,Half-Life,EizanAratoFujimaki,True,0
2,70,Half-Life,GamerFag,True,0
3,70,Half-Life,76561198020928326,True,0
4,70,Half-Life,Bluegills,True,2
5,70,Half-Life,76561198071955492,True,0


In [27]:
# Agrupamos el dataframe por user_id para contar el número de juegos puntuados por usuarios.
conteo = df_features.groupby("user_id").size()

# Seleccionamos solo los usuarios que hayan dejado reviews a al menos 5 juegos
df_features = df_features.loc[df_features["user_id"].isin(conteo[conteo >= 5].index), :]
df_features.head()


,item_id,item_name,user_id,recommend,sentiment_analysis
2,70,Half-Life,GamerFag,True,0
5,70,Half-Life,76561198071955492,True,0
8,70,Half-Life,meeeedie,True,1
10,70,Half-Life,KewlKatzz,True,2
13,70,Half-Life,76561197994404698,True,1


* Creamos una funcion para aplicar el puntaje del rating según el criterio establecido.

In [28]:
def get_rating(sentiment_analysis, recommend):
  """
  Devuelve el rating de acuerdo con el criterio especificado.

  Args:
    sentiment_analysis: El valor del análisis de sentimiento, que puede ser 0, 1 o 2.
    recommend: El valor de la recomendación, que puede ser True o False.

  Returns:
    EL rating, que es un entero entre 1 y 5.
  """
  rating = max(1, min(5, 2 * sentiment_analysis + (1 if recommend else 0)))

  return rating


In [29]:
# Aplicamos la función get_rating al df_features
ratings = df_features.apply(lambda row: get_rating(row['sentiment_analysis'], row['recommend']), axis=1)

# Creamos el dataframe df_ratings con la nueva columna de ratings
df_ratings = df_features[['item_id', 'item_name', 'user_id']].assign(rating=ratings)
df_ratings.head()

,item_id,item_name,user_id,rating
2,70,Half-Life,GamerFag,1
5,70,Half-Life,76561198071955492,1
8,70,Half-Life,meeeedie,3
10,70,Half-Life,KewlKatzz,5
13,70,Half-Life,76561197994404698,3


* Observamos la distribución de los ratings

In [30]:
df_ratings.groupby(["rating"])["user_id"].count()

rating
1    1920
2     185
3    1680
4     284
5    4239
Name: user_id, dtype: int64

* Aplicamos `StandardScaler` para normalizar los ratings, escalando los valores de rating para que tengan una media de 0 y una desviación estándar de 1, lo que es útil para los sistemas de recomendación basados en filtro colaborativo como el nuestro que utilizan la similitud entre los usuarios o los elementos para hacer recomendaciones. Si los datos no están normalizados, los usuarios o los items con valores más altos pueden tener una influencia desproporcionada en las recomendaciones, lo que puede afectar negativamente la calidad de las recomendaciones.

In [31]:
# Inicializamos StandardScaler
scaler = StandardScaler()

# Creamos un array con los ratings
rating_array = df_ratings['rating'].values.reshape(-1, 1)

# Normalizamos
normalized_rating  =scaler.fit_transform(rating_array)

# Creamos un df con los ratings normalizados
df_norm = df_ratings.copy()
df_norm['rating'] = normalized_rating
df_norm.head()

,item_id,item_name,user_id,rating
2,70,Half-Life,GamerFag,-1.565118
5,70,Half-Life,76561198071955492,-1.565118
8,70,Half-Life,meeeedie,-0.347209
10,70,Half-Life,KewlKatzz,0.870699
13,70,Half-Life,76561197994404698,-0.347209


### 4.2 Creando la matriz usuarios/ratings

* Crearemos una matriz en la que cruzamos todos los usuarios con todos los juegos. Para esto creamos un DataFrame que contiene los `user_id` como indices, los `item_name` como columnas y `rating` valores.

In [32]:
df_matrix = df_norm.pivot_table(index=['user_id'], columns=['item_name'], values='rating').fillna(0)
df_matrix

item_name,100% Orange Juice,8BitMMO,A Bird Story,A Story About My Uncle,APB Reloaded,ARK: Survival Evolved,ARMA: Cold War Assault,Ace of Spades: Battle Builder,AdVenture Capitalist,Age of Empires II HD,...,Worms Revolution,XCOM: Enemy Unknown,XCOM® 2,Yet Another Zombie Defense,You Have to Win the Game,Zombie Army Trilogy,how do you Do It?,the static speaks my name,theHunter Classic,theHunter: Primal
user_id,,,,,,,,,,,,,,,,,,,,,
-SEVEN-,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
091263,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1011001,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
12345678901234567890123456567890,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.565118,0.0
1234567io9872345678765432,0.0,0.0,0.0,0.0,-0.956164,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zachwgtv,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
zakbot,0.0,0.0,0.0,0.0,0.000000,-1.565118,0.0,-0.956164,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
zaukster,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [33]:
df_matrix.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1391 entries, -SEVEN- to zyr0n1c
Columns: 548 entries, 100% Orange Juice to theHunter: Primal
dtypes: float64(548)
memory usage: 5.8+ MB


### 4.3 Sparcity

* Para calcular la "Sparcity" de nuestra matriz empleamos la siguiente función:

In [34]:
def get_sparsity(df):
  """
  Devuelve el sparsity de una matriz como df.

  Args:
    df: La matriz en la que se desea calcular el sparsity.

  Returns:
    El sparsity de la matriz `df`.
  """

  num_zeros = (df == 0).sum()
  num_elements = df.size

  sparsity = (1 - num_zeros.sum() / num_elements)*100

  return f'Sparsity: {round(sparsity, 2)}%'

In [35]:
get_sparsity(df_matrix)

'Sparsity: 1.09%'

* Observamos que la matriz es muy dispersa. Con el fin de reducir la memoria utilizada y mejorar la eficiencia en el manejo de los datos emplearemos el módulo `sparse` de la biblioteca `SciPy`. Esto nos permitirá que en lugar de almacenar todos los valores de la matriz, solo se almacenen los valores que no son ceros, lo que puede ahorrar espacio de almacenamiento y mejorar el rendimiento de los cálculos.

In [36]:
matrix_sparse = sp.sparse.csr_matrix(df_matrix.values)
matrix_sparse

<1391x548 sparse matrix of type '<class 'numpy.float64'>'
	with 8308 stored elements in Compressed Sparse Row format>

### 4.4 Train y Test set split

* Separamos en train y test para poder medir la calidad de nuestras recomendaciones.

In [37]:
ratings = df_matrix.values

In [38]:
ratings_train, ratings_test = train_test_split(ratings, test_size = 0.3, random_state=123)
print(ratings_train.shape)
print(ratings_test.shape)

(973, 548)
(418, 548)


### 4.5 Matriz de Similitud: Similitud de coseno

* Creamos dos matrices de similitud utilizando la similitud del coseno para medir la similitud entre los juegos `item_sim_matrix` y entre los usuarios `user_sim_matrix`.

In [39]:
user_sim_matrix = cosine_similarity(matrix_sparse)
print(user_sim_matrix.shape)

(1391, 1391)


In [40]:
item_sim_matrix = cosine_similarity(matrix_sparse.T)
print(item_sim_matrix.shape)

(548, 548)


In [41]:
#item similarity dataframe
df_user_sim = pd.DataFrame(user_sim_matrix, index = df_matrix.index, columns = df_matrix.index)
df_user_sim.head()

user_id,-SEVEN-,091263,1011001,12345678901234567890123456567890,1234567io9872345678765432,12779,131312,1337lolroflmao,1626466724893520,17101710,...,yoshipowerz,yotuic,you_re_ded,youngbenaffleck,zaaikbr,zachwgtv,zakbot,zaukster,zayyntt,zyr0n1c
user_id,,,,,,,,,,,,,,,,,,,,,
-SEVEN-,1.000000,-0.230133,-0.411551,0.345894,-0.234051,0.0,0.0,-0.166343,0.0,0.0,...,0.000000,-0.270148,0.122628,0.037896,0.000000,0.0,0.084326,0.0,0.000000,0.103689
091263,-0.230133,1.000000,0.202183,-0.231632,0.183274,0.0,0.0,0.130255,0.0,0.0,...,0.113761,0.132716,-0.452289,0.000000,0.269786,0.0,-0.056470,0.0,-0.351671,0.000000
1011001,-0.411551,0.202183,1.000000,-0.259880,0.041630,0.0,0.0,0.146140,0.0,0.0,...,0.000000,0.297801,-0.092134,-0.125551,-0.168389,0.0,-0.063357,0.0,0.000000,-0.214538
12345678901234567890123456567890,0.345894,-0.231632,-0.259880,1.000000,-0.235576,0.0,0.0,-0.167427,0.0,0.0,...,0.000000,-0.341179,0.105554,0.000000,0.000000,0.0,0.072585,0.0,0.000000,0.000000
1234567io9872345678765432,-0.234051,0.183274,0.041630,-0.235576,1.000000,0.0,0.0,0.132473,0.0,0.0,...,0.000000,0.216126,-0.083517,-0.042733,0.000000,0.0,-0.057431,0.0,-0.079344,0.077550


In [42]:
df_user_sim.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1391 entries, -SEVEN- to zyr0n1c
Columns: 1391 entries, -SEVEN- to zyr0n1c
dtypes: float64(1391)
memory usage: 14.8+ MB


In [43]:
#user similarity dataframe
df_items_sim = pd.DataFrame(item_sim_matrix, index = df_matrix.columns, columns = df_matrix.columns)
df_items_sim

item_name,100% Orange Juice,8BitMMO,A Bird Story,A Story About My Uncle,APB Reloaded,ARK: Survival Evolved,ARMA: Cold War Assault,Ace of Spades: Battle Builder,AdVenture Capitalist,Age of Empires II HD,...,Worms Revolution,XCOM: Enemy Unknown,XCOM® 2,Yet Another Zombie Defense,You Have to Win the Game,Zombie Army Trilogy,how do you Do It?,the static speaks my name,theHunter Classic,theHunter: Primal
item_name,,,,,,,,,,,,,,,,,,,,,
100% Orange Juice,1.000000,0.0,0.0,0.0,0.0,0.000000,0.0,-0.016656,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.193398,0.0,0.0,0.0
8BitMMO,0.000000,1.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
A Bird Story,0.000000,0.0,1.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
A Story About My Uncle,0.000000,0.0,0.0,1.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
APB Reloaded,0.000000,0.0,0.0,0.0,1.0,-0.012329,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zombie Army Trilogy,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.108056,0.000000,1.0,0.000000,0.0,0.0,0.0
how do you Do It?,0.193398,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,1.000000,0.0,0.0,0.0
the static speaks my name,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,-0.103743,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,1.0,0.0,0.0


In [44]:
df_items_sim.info()

<class 'pandas.core.frame.DataFrame'>
Index: 548 entries, 100% Orange Juice to theHunter: Primal
Columns: 548 entries, 100% Orange Juice to theHunter: Primal
dtypes: float64(548)
memory usage: 2.3+ MB


### 4.6 Error cuadrático medio (MSE)

* Calculamos el MSE para test y train:

In [45]:
#separar las filas y columnas de train y test
user_sim_matrix_train = user_sim_matrix[0:973,0:973]
user_sim_matrix_test = user_sim_matrix[973:1391,973:1391]

print(user_sim_matrix_train.shape)
print(user_sim_matrix_test.shape)

(973, 973)
(418, 418)


In [46]:
# Realizamos las predicciones para el test set
users_predictions = user_sim_matrix_train.dot(ratings_train) / np.array([np.abs(user_sim_matrix_train).sum(axis=1)]).T

# Calculamos el MSE de train
mse = mean_squared_error(users_predictions, ratings_train)
mse

0.010136188118582391

In [47]:
# Realizamos las predicciones para el test set
users_predictions_test = user_sim_matrix.dot(ratings) / np.array([np.abs(user_sim_matrix).sum(axis=1)]).T
users_predictions_test = users_predictions_test[973:1391,:]

# Calculamos el MSE de test
mse = mean_squared_error(users_predictions_test, ratings_test)
mse

0.011919160923170965

* Notamos que para el set de train y test el MSE es bastante cercano. Un indicador de que nuestro modelo tiene buenas predicciones, pues lo contrario sería si el MSE en test fuera 2 veces más (ó la mitad) del valor del MSE de train.

## 5 Carga

Finalmente, en esta sección cargamos nuestros datos transformados para los endpoints que se consumirán en la API a su destino final. Optamos por almacenarlos en formato parquet con compresion snappy para reducir su tamaño de almacenamiento.

In [49]:
# Nombres correspondientes a cada DataFrame
dfs = [df_pseudo_db1, df_pseudo_db2, df_user_sim, df_items_sim, df_matrix, df_user_reviews]
names = ['pseudo-db1.parquet', 'pseudo-db2.parquet', 'user_sim.parquet', 'items_sim.parquet', 'matrix.parquet', 'user_sentiment_analysis.parquet']

for dfs, n in zip(dfs, names):
    # Definimos la ruta del directorio
    #folder_path = f'../data/processed/'
    folder_path = f'data/processed/'


    # Verificamos si el folder_path existe
    if not os.path.exists(folder_path):
        # Si no existe, lo creamos
        os.makedirs(folder_path)

    # Definimos la ruta completa del archivo
    path = os.path.join(folder_path, n)

    # Guardamos el DataFrame como un archivo parquet
    dfs.to_parquet(path, engine='pyarrow', compression='snappy')

    print(f"'{n}' fue guardado correctamente en '{folder_path}'")

'pseudo-db1.parquet' fue guardado correctamente en 'data/processed/'
'pseudo-db2.parquet' fue guardado correctamente en 'data/processed/'
'user_sim.parquet' fue guardado correctamente en 'data/processed/'
'items_sim.parquet' fue guardado correctamente en 'data/processed/'
'matrix.parquet' fue guardado correctamente en 'data/processed/'
'user_sentiment_analysis.parquet' fue guardado correctamente en 'data/processed/'
